In [ ]:
import pandas as pd
import numpy as np

In [ ]:
token = "token"
!git clone https://{token}@github.com/Raghav-Sahni/Hope_speech.git

fatal: destination path 'Hope_speech' already exists and is not an empty directory.


In [ ]:
df_train = pd.read_csv("/content/Hope_speech/Data/PreprocessedData/english_train_preprocess.csv")
df_test = pd.read_csv("/content/Hope_speech/Data/PreprocessedData/english_test_preprocess.csv")
df_dev = pd.read_csv("/content/Hope_speech/Data/PreprocessedData/english_dev_preprocess.csv")

In [ ]:
x_train  = df_train.preprocessed_text.values
y_train = df_train.label.values
x_test  = df_test.preprocessed_text.values
y_test = df_test.label.values

In [ ]:
class_names = list(set(y_train))

In [ ]:
#!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 981 kB 64.0 MB/s 
     |████████████████████████████████| 265 kB 75.7 MB/s 
     |████████████████████████████████| 3.8 MB 64.1 MB/s 
     |████████████████████████████████| 1.3 MB 61.4 MB/s 
     |████████████████████████████████| 468 kB 74.2 MB/s 
     |████████████████████████████████| 182 kB 51.4 MB/s 
     |████████████████████████████████| 880 kB 67.3 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=72059643c198a03e4597737d2995782d3c1dfd7db0c506d665c55a8aad88aa4b
  Stored in directory: /root/.cache/pip/wheels/03/2c/bd/18821c5ecd2a95cc5d6fc61bb08db2f70dd2c952b46fbe8d7b
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=97b3ef9f83208952221e306

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=class_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 46
	99percentile : 86


/usr/local/lib/python3.8/dist-packages/ktrain/utils.py:737: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 16
	95percentile : 45
	99percentile : 81


In [ ]:
#find best learning rate (no need to run)
learner.lr_find(show_plot=True, max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Epoch 1/2
   4/3793 [..............................] - ETA: 19:54:09 - loss: 1.1967 - accuracy: 0.0000e+00


KeyboardInterrupt



In [ ]:
learner.fit_onecycle(8e-5, 1)



begin training using onecycle policy with max lr of 8e-05...
3794/3794 [==============================] - 1342s 351ms/step - loss: 0.2303 - accuracy: 0.9204 - val_loss: 0.1835 - val_accuracy: 0.9252


In [ ]:
learner.validate(class_names=t.get_classes())

89/89 [==============================] - 19s 203ms/step
                 precision    recall  f1-score   support

    Hope_speech       0.65      0.35      0.45       250
Non_hope_speech       0.94      0.98      0.96      2593
    not-English       0.00      0.00      0.00         3

       accuracy                           0.93      2846
      macro avg       0.53      0.44      0.47      2846
   weighted avg       0.91      0.93      0.91      2846



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  87,  163,    0],
       [  47, 2546,    0],
       [   0,    3,    0]])

In [ ]:
#the prediction that was most wrong
learner.view_top_losses(n=1, preproc=t)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.predict(x_test[0])

In [ ]:
#!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 31.4 MB/s 
     |████████████████████████████████| 133 kB 76.8 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=a8a166515be1a223262238cf1662f4c0dbb36c1b6abe98969a4127647081081e
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incomp

In [ ]:
predictor.explain(x_test[0])

79/79 [==============================] - 3s 32ms/step


In [ ]:
predictor.save('bert_hope')

In [ ]:
reloaded_predictor = ktrain.load_predictor('bert_hope')